In [1]:
import numpy as np
import pandas as pd
import math
from sympy import primerange
from scipy.stats import entropy
from fractions import Fraction

# --- Constants ---
phi = (1 + math.sqrt(5)) / 2
alpha = 1 / 137.035999  # Fine-structure constant (scaled for match)
pi = math.pi
sqrt2 = math.sqrt(2)
e = math.e

# Fibonacci word string (symbolic compression)
def fibonacci_word(n):
    a, b = "0", "01"
    while len(b) < n:
        a, b = b, b + a
    return b[:n]

# Convert binary-like string to decimal float (sliding resolution)
def binary_to_decimal_windowed(bitstring, precision=10):
    window_values = []
    for i in range(len(bitstring) - precision):
        window = bitstring[i:i+precision]
        val = sum(int(b) / (2**(j+1)) for j, b in enumerate(window))
        window_values.append(val)
    return window_values

# Symbolic neighborhood pattern matcher
def entropy_over_windows(values, window_size=5):
    entropy_vals = []
    for i in range(len(values) - window_size):
        window = values[i:i+window_size]
        freqs = np.histogram(window, bins=10, range=(0, 1), density=True)[0]
        if np.any(freqs):
            entropy_vals.append(entropy(freqs, base=2))
        else:
            entropy_vals.append(0)
    return entropy_vals

# Compare digit match to known constants at each window
def match_digits_to_constants(values, constants, precision=10):
    results = {name: [] for name in constants}
    for v in values:
        for name, target in constants.items():
            str_v = f"{v:.{precision+2}f}".split(".")[1][:precision]
            str_t = f"{target:.{precision+2}f}".split(".")[1][:precision]
            match_count = sum(1 for a, b in zip(str_v, str_t) if a == b)
            results[name].append(match_count)
    return results

# Main execution
constants = {
    "alpha": alpha,
    "pi": pi,
    "phi": phi,
    "sqrt2": sqrt2,
    "e": e
}

fib_string = fibonacci_word(150)
decimal_windows = binary_to_decimal_windowed(fib_string, precision=12)
entropy_window_vals = entropy_over_windows(decimal_windows, window_size=8)
match_scores = match_digits_to_constants(decimal_windows, constants, precision=8)

# Assemble dataframe
df = pd.DataFrame({
    "decimal_val": decimal_windows,
    "entropy": entropy_window_vals + [None] * (len(decimal_windows) - len(entropy_window_vals))
})
for name, scores in match_scores.items():
    df[f"match_{name}"] = scores

from IPython.display import display
display(df)


,decimal_val,entropy,match_alpha,match_pi,match_phi,match_sqrt2,match_e
0,0.290039,2.25,0,0,2,1,0
1,0.580322,2.25,0,1,0,1,0
2,0.160645,2.25,0,1,0,1,0
3,0.321533,2.25,0,2,2,1,0
4,0.643066,2.25,0,1,2,0,0
...,...,...,...,...,...,...,...
133,0.161133,NaN,0,3,1,0,1
134,0.322510,NaN,0,1,0,1,0
135,0.645020,NaN,0,1,2,2,0
136,0.290039,NaN,0,0,2,1,0
